## **SVM Full Name Model**



In this we have create support vector classification model using full name as token. The support vector machine is a type of algorithm determining the boundary between vector 
belonging to specific class and vectors not belonging to it.The best-fitting line is called 
hyperplane, which divides the space into a given number of planes.SVM algorithm is implemented using kernel. The kernel changes a input data 
space into required form.It take two data point (a and b) and 
calculate a distance score. The closer the point are higher the distance. The data points are them 
mapped in a kernel function (φ((a, b)) = (a, b, a2 + b2)) . This helps in transforming the 
data point in higher dimensional aping

## Data processing

Here We import the ethnicity data from the file and select the relevant columns. We converted labels into numeric and clean the names by removing suffixes and special characters. Cleaning data is essential as it can create wrong models.

**vectorising text** : It is important part to create the token and vocabulary from the words.We have use countvectorizer. It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text. Basically we converted each name into a single vector.



In [ ]:
# Importing librairies
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import re
from sklearn.model_selection import KFold
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
# read data , and choose columns and get info about data
df = pd.read_csv("ethnicity collection data11-asian specific.csv")
df.shape
df.head()
df.groupby('ethinicity')['full name'].size()
df =df[['full name','ethinicity']]

# convert the columns into lower case and remove na
df['full name'] = df['full name'].str.lower()
df['ethinicity']=df['ethinicity'].str.lower()
df['ethinicity']=df['ethinicity'].map({'asian-indian':1,'black non hispanic':2,'hispanic':3 , 'white non hispanic':4,'asian-east':5})
df = df.dropna()
df
# clean text, remove special charaters, numbers,suffix(Full name),  
def cleaning(text):
    only_words = re.sub('([^A-Za-z ]+)|(^dr\.)|(^dr )|(^mr\.)|(^mr )|(^prof\.)|(^adv\. )',' ',text )
    return only_words

df['full_name_cleaned']=df['full name'].apply(cleaning)
df
#save data in new file 
df.to_csv('data.csv', encoding='utf-8', index=False)
#load new data
cleaned_data = pd.read_csv("data.csv", encoding = "ISO-8859-1")
pd.set_option('display.max_colwidth', None)
#convert lables 
data = cleaned_data.full_name_cleaned
target = cleaned_data.ethinicity

# vectorise tex - ngram 2,3,4 ,tf-idf/count vectoriser
countvect = CountVectorizer(ngram_range=(1,2)) 
name = countvect.fit_transform(data)
a =countvect.get_feature_names()
len(a)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/

860086

Here to create a model we have done following steps :



1.   We have used cross- validation for comparing the model and select the best-trained model. It presents 
the general idea of how the model will perform on unseen data. The result of cross-validation
is generally the output of a model that is less biased or less optimistic.We specifically used k-fold technique with k = 10.
2.   We have used SOMTE for oversampling the data. Our data was not balance which can create a problem of bias. To avoide that we used SMOTE.
3. We split the data into train-test and ran the model.



In [ ]:
# model fitting and cross validation k fold - 10 
# we have also done oversamlpling using SMOTE

def Cross_validation(data, tagret, countvect, clf_cv, model_name): #Performs cross-validation 

    kf = KFold(n_splits=10, shuffle=True, random_state=1) # 10-fold cross-validation
    scores=[]
    data_train_list = []
    targets_train_list = []
    data_test_list = []
    targets_test_list = []
    iteration = 0
    print("Performing cross-validation for {}...".format(model_name))
    for train_index, test_index in kf.split(data):
        iteration += 1
        print("Iteration ", iteration)

        # Split train and text data.
        data_train_cv, targets_train_cv = data[train_index], target[train_index] 
        data_test_cv, targets_test_cv = data[test_index], target[test_index] 
        data_train_list.append(data_train_cv) 
        data_test_list.append(data_test_cv) 
        targets_train_list.append(targets_train_cv) 
        targets_test_list.append(targets_test_cv)
        
        # learning vocabulary of training set
        countvect.fit(data_train_cv.values.astype('U')) 
        data_train_countvect_cv = countvect.transform(data_train_cv.values.astype('U'))
        print(data_train_countvect_cv.shape)
        print(targets_train_cv.shape)

        #balancing Trainign dataset for each iteration 
        print("Number of observations in each class before oversampling (training data): \n", pd.Series(targets_train_cv).value_counts())
        smote = SMOTE(random_state = 101)
        data_train_countvect_cv,targets_train_cv = smote.fit_sample(data_train_countvect_cv,targets_train_cv)
        print("Number of observations in each class after oversampling (training data): \n", pd.Series(targets_train_cv).value_counts())

        #print shape of train and test data
        print("Shape of training data: ", data_train_countvect_cv.shape)
        # converting test data using countvectoriser 
        data_test_countvect_cv = countvect.transform(data_test_cv.values.astype('U'))
        print("Shape of test data: ", data_test_countvect_cv.shape)
        clf_cv.fit(data_train_countvect_cv, targets_train_cv) # Fitting model
        score = clf_cv.score(data_test_countvect_cv, targets_test_cv) # Calculating accuracy
        scores.append(score) # appending cross-validation accuracy for each iteration
    print("List of cross-validation accuracies for {}: ".format(model_name), scores)
    mean_accuracy = np.mean(scores)
    print("Mean cross-validation accuracy for {}: ".format(model_name), mean_accuracy)
    print("Best cross-validation accuracy for {}: ".format(model_name), max(scores))

    #finding best cross-validation for best set
    max_acc_index = scores.index(max(scores)) #
    max_acc_data_train = data_train_list[max_acc_index]
    max_acc_data_test = data_test_list[max_acc_index]
    max_acc_targets_train = targets_train_list[max_acc_index] 
    max_acc_targets_test = targets_test_list[max_acc_index] 

    return mean_accuracy, max_acc_data_train, max_acc_data_test, max_acc_targets_train, max_acc_targets_test

def c_matrix(max_acc_data_train, max_acc_data_test, max_acc_targets_train, max_acc_targets_test, countvect, target, clf, model_name): #### Creates Confusion matrix for SVC
    countvect.fit(max_acc_data_train.values.astype('U'))
    max_acc_data_train_countvect = countvect.transform(max_acc_data_train.values.astype('U'))
    max_acc_data_test_countvect = countvect.transform(max_acc_data_test.values.astype('U'))
    clf.fit(max_acc_data_train_countvect, max_acc_targets_train) # Fitting SVC
    targets_pred = clf.predict(max_acc_data_test_countvect) # Prediction on test data
    #  printing confusion mattrix
    conf_mat = classification_report(max_acc_targets_test, targets_pred)
    print(conf_mat)

# run model
SVC_clf = LinearSVC() 
SVC_mean_accuracy, max_acc_data_train, max_acc_data_test, max_acc_targets_train, max_acc_targets_test = Cross_validation(data, target, countvect, SVC_clf, "SVC") # SVC cross-validation
c_matrix(max_acc_data_train, max_acc_data_test, max_acc_targets_train, max_acc_targets_test, countvect, target, SVC_clf, "SVC") # SVC confusion matrix

# Model saving
def SVC_Save(data, target, countvect):
    countvect.fit(data.values.astype('U')) # learn vocabulary of entire data
    data_countvect = countvect.transform(data.values.astype('U'))
    # saving vocabulary
    pd.DataFrame.from_dict(data=dict([word, i] for i, word in enumerate(countvect.get_feature_names())), orient='index').to_csv('vocabulary_SVC.csv', header=False)
    print("Shape of countvectoriser matrix for saved SVC Model: ", data_countvect.shape)
    clf = LinearSVC().fit(data_countvect, target)
    joblib.dump(clf, 'svc.sav')

#saving model
SVC_Save(data, target, countvect)

Performing cross-validation for SVC...
Iteration  1
(560904, 790226)
(560904,)
Number of observations in each class before oversampling (training data): 
 4.0    374214
1.0     85482
3.0     49811
2.0     37902
5.0     13495
Name: ethinicity, dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Number of observations in each class after oversampling (training data): 
 1.0    374214
5.0    374214
4.0    374214
3.0    374214
2.0    374214
dtype: int64
Shape of training data:  (1871070, 790226)
Shape of test data:  (62323, 790226)
Iteration  2
(560904, 789900)
(560904,)
Number of observations in each class before oversampling (training data): 
 4.0    374124
1.0     85736
3.0     49659
2.0     37962
5.0     13423
Name: ethinicity, dtype: int64
Number of observations in each class after oversampling (training data): 
 1.0    374124
5.0    374124
4.0    374124
3.0    374124
2.0    374124
dtype: int64
Shape of training data:  (1870620, 789900)
Shape of test data:  (62323, 789900)
Iteration  3
(560904, 789837)
(560904,)
Number of observations in each class before oversampling (training data): 
 4.0    374394
1.0     85395
3.0     49696
2.0     37972
5.0     13447
Name: ethinicity, dtype: int64
Number of observations in each class after oversampling (training data): 
 1.0    374394
5

Our model was created using vocabulary of 790302 words. We observed here that our model give us accuracy of 0.90 with recall score of 0.98 for indian asian ethnicity. We saved that model and vocabulary for further use.

# Deploying model



To deploy this model on unseen data of the company, we import the data and the saved model and vocabulary. We cleaned the data and removed unwanted data that is not classified as human names (e.g., LLC, trust, ltd .etc). We also removed suffixes and special characters to get a better result. We ran the model on unseen data and checked the results.

In [ ]:
# DEPLOYMENT OF SVC MODEL ON COMPANY DATA

#Import libraries
import re, nltk
import numpy as np
import pandas as pd
import csv
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer

# Load data and info, select the relevant column
df = pd.read_csv(DCourse BAresearchTest datamain1.csv)
df.shape
df.head()
df =df[['prop_owner_name', 'IsSelectedforExport']]

# convert the columns into lower case and remove na
df['prop_owner_name'] = df['prop_owner_name'].str.lower()
df = df.dropna()
# cleaning unwanted data
df = df[~(df.prop_owner_name.str.contains('llc'))]
df = df[~(df.prop_owner_name.str.contains('ltd'))]
df = df[~(df.prop_owner_name.str.contains('estate'))]
df = df[~(df.prop_owner_name.str.contains('trust'))]

# divide two name into seperate column
new=df['prop_owner_name'].str.split(&,n = 1, expand = True)
df['name1']= new[0]
df['name2']= new[1]

 
# clean text, remove special charaters, numbers,suffix(Full name),  
def cleaning(text)
    only_words = re.sub('([^A-Za-z ]+)|(^dr.)|(^dr )|(^mr.)|(^mr )|(^prof.)|(^adv. )',' ',text )
    return only_words

df['name1']=df['name1'].apply(cleaning)

# loading model and vocabulary
model = joblib.load('svc.sav')
vocabulary_model = pd.read_csv('vocabulary_SVC.csv', header=None)

#converting vocabulary in dictionary
vocabulary_model_dict = {}
for i, word in enumerate(vocabulary_model[0])
   vocabulary_model_dict[word] = i
countvect = CountVectorizer(ngram_range=(1,2),vocabulary = vocabulary_model_dict) 

#coberting new names into countvector
new_name_countvect = countvect.fit_transform(df['name1'])

#applying model and prediciting
targets_pred = model.predict(new_name_countvect)
df['predicited_ethnicity'] = targets_pred

# saving prediction to new file
df.to_csv('predicted_name.csv', encoding='utf-8', index=False)
